## Imports

In [1]:
import os
import sys
import glob
import scipy
import mlflow
import pickle
import shutil
import logging
import datetime
import argparse
import commentjson
import numpy as np
from bunch import Bunch
import tensorflow as tf
from pathlib import Path
from random import randint
from collections import OrderedDict
from tensorflow.keras import metrics
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Input
from tensorflow.keras.layers import Dropout,  Activation, LeakyReLU, AveragePooling1D



2023-07-05 13:17:28.840896: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-05 13:17:28.868967: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-05 13:17:28.869655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 13:17:29.353028: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Define logger

In [2]:
logger = logging.getLogger("logger")

### Config class definition

In [3]:
CONFIG_VERBOSE_WAIVER = ['save_model', 'tracking_uri', 'quiet', 'sim_dir', 'train_writer', 'test_writer', 'valid_writer']
class Config(Bunch):
    """ class for handling dicrionary as class attributes """

    def __init__(self, *args, **kwargs):
        super(Config, self).__init__(*args, **kwargs)

    def print(self):
        line_len = 122
        line = "-" * line_len
        logger.info(line + "\n" +
              "| {:^35s} | {:^80} |\n".format('Feature', 'Value') +
              "=" * line_len)
        for key, val in sorted(self.items(), key= lambda x: x[0]):
            if isinstance(val, OrderedDict):
                raise NotImplementedError("Nested configs are not implemented")
            else:
                if key not in CONFIG_VERBOSE_WAIVER:
                    logger.info("| {:35s} | {:80} |\n".format(key, str(val)) + line)
        logger.info("\n")



### define arguments

In [4]:
def get_args(argv):
    argparser = argparse.ArgumentParser(description=__doc__)
    argparser.add_argument('--config', default=None, type=str, help='path to config file')
    argparser.add_argument('--seed', default=None, type=int, help='randomization seed')
    argparser.add_argument('--exp_name', default=None, type=int, help='Experiment name')
    argparser.add_argument('--num_targets', default=None, type=int, help='Number of simulated targets')
    argparser.set_defaults(quiet=False)
    args, unknown = argparser.parse_known_args(argv)
    #args = argparser.parse_args()

    return args


### Read the config file

In [5]:
def read_json_to_dict(fname):
    """ read json config file into ordered-dict """
    fname = Path(fname)
    with fname.open('rt') as handle:
        config_dict = commentjson.load(handle, object_hook=OrderedDict)
        return config_dict
    

In [6]:
def read_config(args):
    """ read config from json file and update by the command line arguments """
    if args.config is not None:
        json_file = args.config
    else:
        json_file = "/home/leshkar/Desktop/BGU/configs/config.json"  # Replace with your default config file path

    config_dict = read_json_to_dict(json_file)
    config = Config(config_dict)

    for arg in sorted(vars(args)):
        key = arg
        val = getattr(args, arg)
        if val is not None:
            setattr(config, key, val)

    if args.seed is None and config.seed is None:
        
        MAX_SEED = sys.maxsize
        config.seed = randint(0, MAX_SEED)

    return config


### GPU initialization

In [7]:
def gpu_init():
    """ Allows GPU memory growth """

    gpus = tf.config.experimental.list_physical_devices('GPU')
    logger.info("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            logger.info(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            logger.info("MESSAGE", e)


### Logger and tracker

In [8]:
def set_logger_and_tracker(config):
    ''' configure the mlflow tracker:
        1. set tracking location (uri)
        2. configure exp name/id
        3. define parameters to be documented
    '''

    config.exp_name_time = "{}_{}_{}".format(config.exp_name,datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"),config.seed)
    config.tensor_board_dir = os.path.join('..',
                                           'results',
                                           config.exp_name,
                                           config.exp_name_time)

    if not os.path.exists(config.tensor_board_dir):
        os.makedirs(config.tensor_board_dir)


def save_scripts(config,SRC_DIR):
    path = os.path.join(config.tensor_board_dir, 'scripts')
    if not os.path.exists(path):
        os.makedirs(path)
    scripts_to_save = glob.glob('{}/**/*.py'.format(SRC_DIR), recursive=True) + [config.config]
    scripts_to_save = [script for script in scripts_to_save if '{}/results'.format(SRC_DIR) not in script]
    if scripts_to_save is not None:
        for script in scripts_to_save:
            dst_file = os.path.join(path, os.path.basename(script))
            try:
                shutil.copyfile(os.path.join(os.path.dirname(sys.argv[0]), script), dst_file)
            except:
                print()

In [9]:
def print_config(config):
    print('')
    print('#' * 70)
    print('Configurations at beginning of run')
    print('#' * 70)
    for key in config.keys():
        print('{}, {}'.format(key,config['{}'.format(key)]))
    print('')
    print('')

In [18]:
def save_config(config):
    conf = config
    file_path = 'config.pickle'  # Path to the file where you want to save the object
    with open(file_path, 'wb') as file:
        pickle.dump(config, file)


In [19]:
SRC_DIR = os.getcwd()
config_path = '/home/leshkar/Desktop/BGU/configs/config.json'  
args = get_args(config_path)
config = read_config(args)
save_config(config)
print(type(config))
gpu_init()
set_logger_and_tracker(config)
#save_scripts(config,SRC_DIR)
#print_config(config)


<class '__main__.Config'>


#### config properties

In [11]:
config_properties = list(config)
config_properties.sort()

for attr_name in config_properties:
    attr_value = config[attr_name]
    print(f'{attr_name}: {attr_value}')


B_chirp: 50000000.0
CBBCE_penalize_interference: False
CBBCE_penalize_margin: 5
CBBCE_penalize_snr_use_geom_space: False
CBBCE_predefined_weight: 0
CBBCE_use_penalize_margin: False
CBBCE_use_penalize_snr: False
CNR_db: 15
FOV: 60
K: 64
L: 10
M_test: 10000
M_train: 10000
M_valid: 1000
N: 64
SCNR_db: -5
SCNR_db_random_choice: False
SCNR_db_random_constant: False
SCNR_db_range: [-5, 10]
SCNRs_eval: [-10, -7.5, -5, -2.5, 0, 2.5, 5, 7.5, 10]
T_PRI: 0.001
T_idle: 5e-05
activation: tanh
activation_sweep_list: ['relu', 'tanh']
additive_noise_std: 1.0
augment_list: []
augment_prob: 0.5
batch_size: 256
batch_size_sweep_list: [32, 64, 128, 256, 512]
beamforming_method_sweep_list: ['MVDR', 'MUSIC', 'MLE']
cfar_guard_cell: [0.1, 0.1, 0.1]
cfar_method: ca
cfar_num_censor_cells_largest: 0.25
cfar_num_censor_cells_largest_sweep_list: [0.1, 0.25, 0.5, 0.75]
cfar_num_censor_cells_smallest: 0.25
cfar_os_order_statistic: 0.5
cfar_os_order_statistic_sweep_list: [0.25, 0.5, 0.75]
cfar_single_param: []
cfar_